In [1]:
import os
import json
import numpy as np
import io
from datetime import datetime
from data_generator import DataGenerator
from trade_env import TraderEnv
from rnn.model_keras import NeuralNetwork
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from sklearn.preprocessing import MinMaxScaler

/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dt = DataGenerator(random=False, first_index=10)
dt.rewind()

In [3]:
# integer encode input data
def onehot_encoded (integer_encoded, char_to_int = 2):
    # one hot encode
    onehot_encoded = list()
    letter = [0 for _ in range(char_to_int)]
    letter[integer_encoded] = 1
    onehot_encoded.append(letter)
    
    return onehot_encoded[0]

In [4]:
last_price = 0
last_time = 0
steps = 100
def get_state(raw_state):
    global last_price
    global last_time
    list = []

    price = raw_state["price"]

    def prepare_orders(orders, price, multi):
        amount = float(orders[0][1])
        for order in orders:
            list.append((float(order[0])/price) * multi)
            list.append(float(order[1])/amount)

    bids = raw_state["bids"][:5]
    asks = raw_state["asks"][:5]
    prepare_orders(bids, price, 1)
    prepare_orders(asks, price, -1)


    if last_price != 0:
        list.extend([price/last_price])
    else:
        list.extend([0])
        
    
    if last_time != 0:
        list.extend([int(state['timestamp'])/last_time])
    else:
        list.extend([0])
        
    last_time = int(state['timestamp'])
    
    y = dt.get_from_index(dt.index + steps)["price"]
    
    if y > price:
        y = onehot_encoded(0)
    #elif y < price: 
    #    y = onehot_encoded(2)
    else:
        y = onehot_encoded(1)
        
    #datetime.fromtimestamp(int(state['timestamp']))
    
    return [list, y]


In [5]:
trainX = []
trainY = []
while len(trainX) < 100000: #(dt.max_steps() - steps*3):
    state = dt.next()
    state = get_state(state)
    trainX.append(state[0])
    trainY.append(state[1])
trainX = np.array(trainX)
trainY = np.array(trainY)
print(trainX.shape)
print(trainY.shape)

(100000, 22)
(100000, 2)


In [6]:
def get_batches(data, features, batch_size, seq_length):
    int_text = np.array(data)
    
    n_batches = int(len(data) / (batch_size * seq_length))

    xdata = np.array(data[: n_batches * batch_size * seq_length])
    
    data = xdata.reshape(-1, seq_length, features)
    
    #step1 = xdata.reshape(batch_size, -1)
    
    #print (step1.shape)
    #data = np.split(step1, n_batches, 1)
    
    return np.array(list(data))

In [7]:
print(trainX.shape)
batch_size = 20
seq_length = 100
xdata = get_batches(trainX, trainX.shape[1], batch_size, seq_length)
print(xdata.shape)

(100000, 22)
(1000, 100, 22)


In [8]:
print(trainY.shape)
ydata = get_batches(trainY, trainY.shape[1], batch_size, seq_length)
#ydata = ydata.reshape(9900, seq_length, 3)
print(ydata.shape)

(100000, 2)
(1000, 100, 2)


In [9]:
y_train_reshaped = []
for i in range(ydata.shape[0]):
    y_train_reshaped.append(ydata[i, -1])

In [10]:
y_train_reshaped = np.array(y_train_reshaped)
print(y_train_reshaped.shape)

(1000, 2)


In [11]:
nn = NeuralNetwork(0.00001, xdata.shape[2], y_train_reshaped.shape[1], xdata.shape[1])

In [12]:
nn.train(xdata, y_train_reshaped)

Train on 700 samples, validate on 300 samples
Epoch 1/150
700/700 [==============================] - 5s 8ms/step - loss: 7.6198 - mean_absolute_error: 0.5098 - acc: 0.4886 - val_loss: 7.1608 - val_mean_absolute_error: 0.4499 - val_acc: 0.5500
Epoch 2/150
700/700 [==============================] - 4s 6ms/step - loss: 7.0925 - mean_absolute_error: 0.4542 - acc: 0.5457 - val_loss: 7.0867 - val_mean_absolute_error: 0.4462 - val_acc: 0.5533
Epoch 3/150
700/700 [==============================] - 4s 5ms/step - loss: 6.7811 - mean_absolute_error: 0.4421 - acc: 0.5586 - val_loss: 7.0266 - val_mean_absolute_error: 0.4484 - val_acc: 0.5500
Epoch 4/150
700/700 [==============================] - 3s 5ms/step - loss: 6.6796 - mean_absolute_error: 0.4619 - acc: 0.5371 - val_loss: 7.1464 - val_mean_absolute_error: 0.4699 - val_acc: 0.5300
Epoch 5/150
700/700 [==============================] - 3s 5ms/step - loss: 7.3541 - mean_absolute_error: 0.4934 - acc: 0.5057 - val_loss: 8.0744 - val_mean_absolute_e

700/700 [==============================] - 3s 5ms/step - loss: 6.5698 - mean_absolute_error: 0.4421 - acc: 0.5586 - val_loss: 7.2255 - val_mean_absolute_error: 0.4764 - val_acc: 0.5200
Epoch 43/150
700/700 [==============================] - 3s 5ms/step - loss: 6.7855 - mean_absolute_error: 0.4582 - acc: 0.5443 - val_loss: 6.8683 - val_mean_absolute_error: 0.4573 - val_acc: 0.5400
Epoch 44/150
700/700 [==============================] - 3s 5ms/step - loss: 6.5963 - mean_absolute_error: 0.4436 - acc: 0.5571 - val_loss: 6.6168 - val_mean_absolute_error: 0.4587 - val_acc: 0.5367
Epoch 45/150
700/700 [==============================] - 3s 5ms/step - loss: 6.4635 - mean_absolute_error: 0.4315 - acc: 0.5743 - val_loss: 6.7957 - val_mean_absolute_error: 0.4552 - val_acc: 0.5400
Epoch 46/150
700/700 [==============================] - 3s 5ms/step - loss: 6.2289 - mean_absolute_error: 0.4215 - acc: 0.5814 - val_loss: 6.7549 - val_mean_absolute_error: 0.4521 - val_acc: 0.5433
Epoch 47/150
700/700 [=

KeyboardInterrupt: 

In [ ]:
dt.index